In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# read word development indicator data
climate = pd.read_csv('../input/world-development-indicators-by-countries/climate.csv', delimiter=',')
gg_emissions =pd.read_csv('../input/world-development-indicators-by-countries/greenhouse_gas_emissions.csv', delimiter=',')
sustainability = pd.read_csv('../input/world-development-indicators-by-countries/sustainability.csv', delimiter=',')
agriculture=pd.read_csv('../input/world-development-indicators-by-countries/argicultural_inputs.csv', delimiter=',')
emissions =pd.read_csv('../input/world-development-indicators-by-countries/emissions.csv', delimiter=',')
rural =pd.read_csv('../input/world-development-indicators-by-countries/rural.csv', delimiter=',')
energy =pd.read_csv('../input/world-development-indicators-by-countries/energy.csv', delimiter=',')
water =pd.read_csv('../input/world-development-indicators-by-countries/freshwater.csv', delimiter=',')

# read footprint data
footprint = pd.read_csv('../input/national-footprint-accounts-2018/NFA 2018.csv', delimiter=',')

# create a tuple for world development indicator data
indicators = (climate, gg_emissions, sustainability, agriculture, emissions, rural, energy, water)

# import functions to build data set
from dataprocess import *

# initialize footprint data
fp=initialize_footprint(footprint)

In [ ]:
#set 2 features
#follow the order climate, gg_emissions, sustainability, agriculture, emissions, rural, energy, water
keyList = [
    [],
    [],
    [3, 4, 9],
    [2,10],
    [10,12],
    [2, 6,12],
    [10,3],
    []
]  


In [ ]:
#build wdi data frame
#Set 1 features
keyList = [
    [],
    [2],
    [3,9,4],
    [8],
    [],
    [],
    [2],
    [3]
] 

wdi_df = build_wdi_df(indicators, keyList, 0)

# use the following metrics to build dataset
metric1 = 'BiocapTotGHA'
metric2 = 'EFConsTotGHA'
dataName = 'total'
year = 2014
countries, data, labels = build_dataset(wdi_df, fp, metric1, metric2, dataName, year)


In [ ]:
# Experiment with different ANN

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

clf = MLPClassifier(hidden_layer_sizes=(16,5),
                    max_iter=5000,
                    activation = 'relu',
                    solver='adam',
                    random_state=1)
scores=[]
estimators = []
train_data = []
train_labels = []
test_data = []
test_labels = []

for k in range(20):
    X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.33)
    clf.fit(X_train, y_train) 
    scores.append(clf.score(X_test, y_test))
    estimators.append(clf)
    train_data.append(X_train)
    train_labels.append(y_train)
    test_data.append(X_test)
    test_labels.append(y_test)
print(scores)

print('Average score:', sum(scores)/len(scores))
print('Standard deviation:', np.std(scores))

In [ ]:
from matplotlib import pyplot as plt

clf = MLPClassifier(hidden_layer_sizes=(16, 5), 
                    max_iter=10,
                    activation = 'relu',
                    solver='adam',
                    random_state=1,
                    warm_start=True)
j = 19
train_scores = []
test_scores =[]

for k in range(1000):
    clf.fit(train_data[j], train_labels[j])
    train_scores.append(clf.score(train_data[j], train_labels[j]))
    test_scores.append(clf.score(test_data[j], test_labels[j]))

ylim=()
train_iter = [k for k in range(10, 10010, 10)]

plt.figure()
plt.title('Learning Curve')
if ylim is not None:
    plt.ylim(*ylim)

plt.plot(train_iter, train_scores, color='r', label='Training')
plt.plot(train_iter, test_scores, color='b', label='Validation')
plt.legend(loc="best")
plt.show()   